In [1]:
from dotenv import load_dotenv
from environment.game import Game
from environment.generators.random_grid_generator import RandomGridGenerator
from environment.grid_visualizer import GridVisualizer
from environment.types import Direction

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")

/home/jovyan/my-conda-envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

1. Create a story

In [1]:
from llm.llm_story_generator import LlmStoryGenerator

story = LlmStoryGenerator().create_new_story(dummy=True)